In [2]:
#pip install pandas_datareader

  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
pwd

'C:\\Users\\Kim'

In [3]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web



In [4]:
def d1(S,K,T,r,sigma):
    '''
    S: Current Stock Price
    K: Strike Price
    T: Time to maturity
    r: Risk-free interest rate
    sigma: Volatility/Standard Deviation of log returns

    returns: volatility weighted return rate
    '''
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))

def d2(S,K,T,r,sigma):
    '''
    S: Current Stock Price
    K: Strike Price
    T: Time to maturity
    r: Risk-free interest rate
    sigma: Volatility/Standard Deviation of log returns

    returns: volatility weighted interest rate
    '''
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

###
def bs_call(S,K,T,r,sigma):
    '''
    S: Current Stock Price
    K: Strike Price
    T: Time to maturity
    r: Risk-free interest rate
    sigma: Volatility/Standard Deviation of log returns

    returns: Black-Sholes Call option price
    '''
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    '''
    S: Current Stock Price
    K: Strike Price
    T: Time to maturity
    r: Risk-free interest rate
    sigma: Volatility/Standard Deviation of log returns

    returns: Black-Sholes Put option price
    '''
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)



In [8]:


###
### Adding more information for additional analysis
def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

###
### THE GREEKS @prestal33 
def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))

   
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))


In [9]:
### Testing Inputs 
stock = 'SPY'
expiry = '12-18-2022'
strike_price = 370

today = datetime.now()
one_year_ago = today.replace(year=today.year-1)

### Testing inputs
df = web.DataReader(stock, 'yahoo', one_year_ago, today)

df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

sigma = np.sqrt(252) * df['returns'].std()
uty = (web.DataReader(
    "^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days / 365
###
### Testing the implied Volatiltiy functions
print("Implied Volatility: " +
      str(100 * call_implied_volatility(bs_call(lcp, strike_price, t, uty, sigma,), lcp, strike_price, t, uty,)) + " %")
print("Implied Volatility: " +
      str(100 * put_implied_volatility(bs_put(lcp, strike_price, t, uty, sigma,), lcp, strike_price, t, uty,)) + " %")

print('The Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma))

Implied Volatility: 15.00000000000001 %
Implied Volatility: 15.00000000000001 %
The Option Price is:  86.34893310518385
